In [14]:
import os
data_dirpath = '/usr2/scratch/fanfic/ao3_harrypotter_text/'
para_dirpath = '/usr2/mamille2/fanfiction-project/data/ao3/harrypotter/'

# Train/dev/test split on Harry Potter

In [50]:
# Load files, get fic IDs

fic_ids = set()

for fname in os.listdir(para_dirpath):
    fic_ids.add(fname.split('_')[0])
    
fic_ids = list(fic_ids)
len(fic_ids)

5968

In [51]:
import numpy as np

# Shuffle
np.random.shuffle(fic_ids)

# Split
train, dev, test = np.split(fic_ids, [int(.8*len(fic_ids)), int(.9*len(fic_ids))])
print(len(train))
print(len(dev))
print(len(test))

4774
597
597


In [38]:
train[:100]

array(['6874807', '8793700', '12902652', '7722616', '15838335', '3458153',
       '7519285', '2378180', '13490070', '12926859', '10794081', '607339',
       '782314', '8435428', '14461707', '11605263', '11256465', '15818769',
       '12173901', '12246120', '2220429', '15069470', '6121357',
       '13279866', '14427993', '11236329', '620111', '9337694', '14962394',
       '1790539', '9140548', '1032054', '8687077', '13850862', '12269997',
       '3889237', '4905367', '10758525', '11611539', '10760664', '8793700',
       '3558710', '689365', '6555610', '2611883', '6685723', '3374381',
       '10889535', '5344055', '11571243', '7606468', '10589658', '585155',
       '7356304', '7407136', '9619868', '1643708', '11996958', '5063110',
       '12540468', '4716347', '5013064', '13486242', '8671843', '10608648',
       '7067200', '10978020', '9472409', '14063925', '5124476', '14161812',
       '15338505', '10494132', '11740872', '7215586', '14981177',
       '15301383', '410533', '10023854', '1

In [47]:
test[:100]

array(['2588804', '15643263', '528487', '6053377', '14920538', '14369391',
       '16367873', '11611539', '15425523', '14753663', '14384850',
       '1019251', '10350147', '11378733', '3595716', '8665645', '2273616',
       '429025', '3502418', '4430621', '13679598', '11721735', '11217111',
       '3383882', '13747413', '12805845', '10374135', '12826443',
       '15216785', '16272431', '8175197', '10361034', '8641240', '2423987',
       '4572786', '374911', '12044190', '4570407', '2370179', '4840652',
       '14188728', '11650530', '9885926', '9500819', '9215741', '7557562',
       '11256399', '4121866', '6895717', '13224567', '11767326',
       '12992835', '11235936', '14229615', '2497955', '9054610',
       '14034414', '5611336', '12020133', '4422662', '7987327', '13176687',
       '892956', '1649498', '1155472', '926514', '1155472', '6866074',
       '4809158', '9895496', '2703395', '8055868', '1155472', '14593281',
       '16105160', '5238752', '6632236', '12540852', '9847772', '82

In [43]:
type(train[0])

numpy.str_

In [52]:
from tqdm import tqdm_notebook as tqdm

for fname in tqdm(os.listdir(para_dirpath)):
    if fname in ['train', 'test', 'dev']:
        continue
    fic_id = fname.split('_')[0]
    
    if fic_id in train:
        os.rename(os.path.join(para_dirpath, fname), os.path.join(para_dirpath, 'train', fname))
    elif fic_id in dev:
        os.rename(os.path.join(para_dirpath, fname), os.path.join(para_dirpath, 'dev', fname))
    elif fic_id in test:
        os.rename(os.path.join(para_dirpath, fname), os.path.join(para_dirpath, 'test', fname))
        
    else:
        print(fic_id)

In [54]:
# Print out train, test, dev fic_ids

with open(os.path.join(para_dirpath, f'train_fic_ids.txt'), 'w') as f:
    for fic_id in sorted(train):
        f.write(f"{fic_id}\n")
        
with open(os.path.join(para_dirpath, f'dev_fic_ids.txt'), 'w') as f:
    for fic_id in sorted(dev):
        f.write(f"{fic_id}\n")
        
with open(os.path.join(para_dirpath, f'test_fic_ids.txt'), 'w') as f:
    for fic_id in sorted(test):
        f.write(f"{fic_id}\n")

# Basic stats on Harry Potter AO3 collection

In [5]:
import pandas as pd

stories = pd.read_csv(os.path.join(data_dirpath, 'stories.csv'))
print(len(stories))
print(stories.columns)

179407
Index(['fic_id', 'title', 'author', 'author_key', 'rating', 'category',
       'fandom', 'relationship', 'character', 'additional tags', 'language',
       'published', 'status', 'status date', 'words', 'comments', 'kudos',
       'bookmarks', 'hits', 'chapter_count', 'series', 'seriespart',
       'seriesid', 'summary', 'preface_notes', 'afterword_notes'],
      dtype='object')


In [8]:
pd.set_option('display.max_colwidth', -1)

In [10]:
stories.loc[:,['hits', 'words', 'additional tags']].head(20)

,hits,words,additional tags
0,2773.0,8389.0,"[""pre-HBP""]"
1,27154.0,15688.0,"[""Alternate Universe"", ""Alternate Universe - Canon Divergence"", ""Episode Tag""]"
2,4880.0,14921.0,"[""Suicide Attempt""]"
3,6690.0,26476.0,"[""Harry Potter is Voldemort's Child"", ""Harry is Harrison Riddle"", ""Three types of people"", ""Two Types of Faunus"", ""Hounds bonded to huntsmen""]"
4,279.0,8861.0,"[""Yona of the Dawn retelling"", ""High Fantasy"", ""Alternate Universe"", ""Spoilers"", ""Yona of the Dawn"", ""myth building"", ""supernatural and magic!"", ""teen for now"", ""no major character deaths (yet)"", ""not super graphic or violent yet either"", ""Unrequited Love"", ""altered ages""]"
5,816.0,257.0,"[""Slash"", ""Justification"", ""Unhealthy Relationships"", ""Dark Arts""]"
6,2517.0,8938.0,"[""Infidelity"", ""Background Het"", ""Anal Sex""]"
7,1323.0,338.0,"[""Potions"", ""Potions Accident"", ""unapologetic fluff"", ""Prelude to Smut""]"
8,272.0,711.0,"[""Best Friends"", ""Implied Sexual Content"", ""are you joking?"", ""What Was I Thinking?"", ""Sexual Humor"", ""WTF"", ""Inspired by Music"", ""Quintuple Drabble"", ""Please Don't Kill Me""]"
9,731.0,1390.0,"[""Ficlet"", ""Angst"", ""Drama""]"


In [12]:
# total number of words
stories['words'].sum()

1601006008.0

## Tag vocabulary

In [25]:
tag_lists = stories['additional tags'].map(lambda x: eval(x.lower()))
len(tag_lists)

179407

In [26]:
all_tags = [x for l in tag_lists for x in l]
len(all_tags)

897364

In [27]:
# empty tags
len([l for l in tag_lists if len(l) > 0])

149465

In [28]:
unique_tags = set(all_tags)
len(unique_tags)

168243

In [32]:
from collections import Counter

tag_counts = Counter(all_tags)

for threshold in [5, 10, 20, 50]:
    print(f'{threshold}: {len([t for t,count in tag_counts.items() if count >= threshold])}')

5: 10737
10: 6365
20: 3881
50: 1910


## Filtering

Filter fics by:
* Minimum #words
* Contain at least 1 tag that occurs more than 50 times
* Minimum #hits (optional)

# Preprocessing

* Figure out if punctuation is messed up (space after full stops)
* spaCy tokenization
* 1 sentence/line, file for each fic